For the sake of the kernel dying even on the 4gig `ml.t2.medium` sagemaker notebook instance, using my [notebook here](https://github.com/namoopsoo/learn-citibike/blob/2020-revisit/notes/2020-06-28-take2.md) , going to use [the notes on class distribution](https://github.com/namoopsoo/learn-citibike/blob/2020-revisit/notes/2020-06-26.md#looking-at-target-class-distribution) to reduce the size

In [1]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1
import fresh.preproc.v2 as pv2

In [2]:
datadir = '/opt/data'
localdir = '/opt/program'


tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv'
                     )#.sample(frac=0.017, random_state=42)
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)

In [3]:
tripsdf.shape

(843416, 15)

In [7]:
x = np.random.choice(np.array(range(20)), size=10, replace=False, 
                 p=[1/200 for _ in range(20)])
sorted(x)

ValueError: probabilities do not sum to 1

In [10]:
a = np.array([.1, .2, .3, .4])
average = .25
b = .25/a
new_norm = np.sum(b)
c = b/new_norm
c

array([0.48, 0.24, 0.16, 0.12])

In [18]:
c = fu.rebalance_proportions(a)
near1 = np.sum(c)
#d = c/near1
#sum(d)
print(c, near1)

[0.48 0.24 0.16 0.12] 1.0000000000000002


In [17]:
# ok , looks like np.random.choice is ok with near1 sum..
x = np.random.choice(np.array([1,2,3,4]), size=4, replace=False, 
                 p=c)
sorted(x)

[1, 2, 3, 4]

### 2020-07-01

#### Testing out dataset rebalancing ..
First here creating a random dataset of a predictable proportion , and verifying I can tweak it..

In [53]:
reload(fu); 
def make_skewed_array(skew):
    array = np.array([1, 2, 3, 4, 5])
    size = array.shape[0]
    new_size = 10000
    newarray = np.random.choice(array, replace=True,
                    size=new_size, p=skew)
    return newarray
    
newarray = make_skewed_array(skew=[.1, .1, .3, .4, .1])
print('skew:', fu.get_proportions(newarray))
print('classes:', list(sorted(set(newarray))))
print('size before', newarray.shape)

_, newy = fu.balance_dataset(newarray, newarray, shrinkage=.5)

print(fu.get_proportions(newy))
print('size after', newy.shape)

skew: {5: 0.0985, 4: 0.399, 3: 0.2995, 1: 0.101, 2: 0.102}
classes: [1, 2, 3, 4, 5]
size before (10000,)
{2: 0.1994, 1: 0.1968, 3: 0.2198, 5: 0.1946, 4: 0.1894}
size after (5000,)
